In [1]:
import pandas as pd
import os
import logging
from typing import List, Dict, Any
from langchain_core.documents import Document

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def build_document(
    row: pd.Series,
    sheet_name: str,
    filename: str,
    content_fields: List[str],
    metadata_fields: Dict[str, str]
) -> Document:
    """通用文档构建函数
    
    Args:
        row: 数据行
        sheet_name: 工作表名称
        filename: 文件名
        content_fields: 需要放入page_content的字段列表
        metadata_fields: 元数据字段映射 {元数据键名: 数据列名}
    
    Returns:
        LangChain Document对象
    """
    # 构建元数据
    metadata = {"来源表格": f"{filename}，{sheet_name}"}
    for meta_key, col_name in metadata_fields.items():
        metadata[meta_key] = str(row[col_name])  # 确保所有值为字符串
    
    # 构建内容
    content_parts = []
    for field in content_fields:
        content_parts.append(f"{field}：{str(row[field])}")
    
    return Document(
        page_content="\n".join(content_parts),
        metadata=metadata
    )

def validate_dataframe(df: pd.DataFrame, required_columns: List[str], sheet_name: str) -> None:
    """验证DataFrame是否包含必需列"""
    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        raise ValueError(
            f"工作表 '{sheet_name}' 缺少必要列: {missing_cols}\n"
            f"现有列: {list(df.columns)}"
        )

def process_excel_to_rag(sheet_configs:dict,file_path: str) -> List[Document]:
    """处理Excel文件生成RAG文档
    
    Args:
        file_path: Excel文件路径
    
    Returns:
        包含所有文档的列表
    """
    # 获取基础文件名（不带路径）
    filename = os.path.basename(file_path)
    documents = []
    # 处理每个工作表
    for sheet_name, config in sheet_configs.items():
        try:
            # 读取数据
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            logger.info(f"成功读取工作表: {sheet_name}, 共 {len(df)} 行")
            
            # 数据验证
            validate_dataframe(df, config['required_columns'], sheet_name)
            
            # 处理每一行
            for _, row in df.iterrows():
                # 跳过关键字段为空的行
                #if pd.isnull(row['病历']):
                    #logger.warning(f"跳过空病历记录（工作表 {sheet_name}）")
                    #continue
                
                # 构建文档
                doc = build_document(
                    row=row,
                    sheet_name=sheet_name,
                    filename=filename,
                    content_fields=config['content_fields'],
                    metadata_fields=config['metadata_fields']
                )
                documents.append(doc)
                
        except Exception as e:
            logger.error(f"处理工作表 {sheet_name} 时出错: {str(e)}")
            raise
    
    logger.info(f"共生成 {len(documents)} 个文档")
    return documents
excel_path_0 = "A-2-5_模型喂养数据-护理病历质控规则.xlsx"
sheet_configs_0 = {
    '汇总': {
        'required_columns': ['性别','年龄','入院诊断','体征单诊断','病历','检查项目','备注','问题控件'],
        'content_fields': ['性别','年龄','入院诊断','体征单诊断','病历','检查项目','备注','问题控件'],
        'metadata_fields': {'性别': '性别', '年龄': '年龄', '入院诊断': '入院诊断', '体征单诊断': '体征单诊断', '病历': '病历', '检查项目': '检查项目', '备注': '备注', '问题控件': '问题控件'}
    }
}

excel_path_1 = "A-2-5-1_模型喂养数据-护理病历质控规则-文字版.xlsx"
sheet_configs_1 = {
    '质控规则': {
        'required_columns': ['项目', '项目检查内容释义','二级标题','三级标题'],
        'content_fields': ['项目', '项目检查内容释义','二级标题','三级标题'],
        'metadata_fields': {'项目': '项目', '项目检查内容释义': '项目检查内容释义', '二级标题': '二级标题', '三级标题': '三级标题'}
    },
    '质控问题': {
        'required_columns': ['质控问题'],
        'content_fields': ['质控问题'],
        'metadata_fields': {'质控问题': '质控问题'}
    },
    '记录单名称': {
        'required_columns': ['病历名称'],
        'content_fields': ['病历名称'],
        'metadata_fields': {
            '病历名称': '病历名称'
        }
    }
}

excel_path_2 = "A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx"
sheet_configs_2 = {
    '时效性': {
        'required_columns': ['病历', '完成时间'],
        'content_fields': ['病历', '完成时间'],
        'metadata_fields': {'病历类型': '病历', '完成时间': '完成时间'}
    },
    '完整性': {
        'required_columns': ['病历', '必填项'],
        'content_fields': ['病历', '必填项'],
        'metadata_fields': {'病历类型': '病历', '必填项': '必填项'}
    },
    '合理性-互补词': {
        'required_columns': ['病历', '互补词条件', '互补控件'],
        'content_fields': ['病历', '互补词条件', '互补控件'],
        'metadata_fields': {
            '病历类型': '病历', 
            '互补词条件': '互补词条件',
            '互补控件': '互补控件'
        }
    },
    '环节质控检查项': {
        'required_columns': ['病历', '环节点', '分类', '检查项'],
        'content_fields': ['病历', '环节点', '分类', '检查项'],
        'metadata_fields': {
            '病历类型': '病历',
            '环节节点': '环节点',
            '分类': '分类',
            '检查项': '检查项'
        }
    }
}

try:
    #rag_docs_0 = process_excel_to_rag(sheet_configs_0,excel_path_0)
    rag_docs_1 = process_excel_to_rag(sheet_configs_1,excel_path_1)
    rag_docs_2 = process_excel_to_rag(sheet_configs_2,excel_path_2)
except Exception as e:
    logger.error(f"处理Excel文件失败: {str(e)}")
    raise
from typing import List, Any
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma  # 改为导入Chroma
from langchain_core.documents import Document

def vectorize_documents(
    all_docs: List[List[Document]],  # 接收二维文档列表
    embedding_model: str = "local",
    vector_db: str = "chroma",
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> Any:
    
    # 展平嵌套文档列表
    combined_docs = [doc for sublist in all_docs for doc in sublist]
    
    # 文档结构验证
    for i, doc in enumerate(combined_docs):
        if not isinstance(doc, Document):
            raise TypeError(f"第 {i} 个元素不是Document对象，实际类型：{type(doc)}")
        if not hasattr(doc, "page_content"):
            raise AttributeError(f"第 {i} 个Document缺少page_content属性")

    # 文本分割
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    splits = text_splitter.split_documents(combined_docs)

    # 本地模型加载（适配您的目录结构）
    embeddings = HuggingFaceEmbeddings(
        model_name="./models/text2vec-base-chinese",
        model_kwargs={'device': 'cuda'},  
        encode_kwargs={'normalize_embeddings': True}
    )
    
    # 创建向量库
    return Chroma.from_documents(splits, embeddings)

# 创建向量库
vector_store = vectorize_documents(
    all_docs=[rag_docs_1, rag_docs_2],
    embedding_model="local",  # 切换模型时修改此参数
    vector_db="chroma",
    chunk_size=1000,
    chunk_overlap=200
)

INFO:__main__:成功读取工作表: 质控规则, 共 316 行
INFO:__main__:成功读取工作表: 质控问题, 共 144 行
INFO:__main__:成功读取工作表: 记录单名称, 共 42 行
INFO:__main__:共生成 502 个文档
INFO:__main__:成功读取工作表: 时效性, 共 6 行
INFO:__main__:成功读取工作表: 完整性, 共 130 行
INFO:__main__:成功读取工作表: 合理性-互补词, 共 65 行
INFO:__main__:成功读取工作表: 环节质控检查项, 共 314 行
INFO:__main__:共生成 515 个文档
/tmp/ipykernel_3736770/4058557662.py:202: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
INFO:datasets:PyTorch version 2.5.1+cu121 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: ./models/text2vec-base-chinese
INFO:chromadb.telemetry.product.posthog:Anonymized teleme

In [ ]:
#召回测试
test_queries = ["护理记录单", "入院评估单"]

for query in test_queries:
    print(f"\n测试查询：'{query}'")
    results = vector_store.similarity_search_with_score(query, k=10)
    
    for i, (doc, score) in enumerate(results):
        print(f"结果 {i+1} (相似度得分：{score:.3f}):")
        print(doc.page_content[:200] + "...")
        print("-" * 50)

In [ ]:
from langchain_openai import ChatOpenAI
import os

# 配置DeepSeek API参数
os.environ["DEEPSEEK_API_KEY"] = "sk-19af4c75b5ba4ddaa864f7328a5e1073"

# 创建Chat模型实例
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=1024,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com/v1"  
)

In [14]:
# 新增依赖
from typing import Tuple, List
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 构建检索增强生成（RAG）系统
def create_qa_chain(vector_store, llm):
    # 中文提示模板
    prompt_template = """根据以下病历质控规则和患者病历，严格按以下要求进行分析：
    1. 逐条列出所有发现的质控问题
    2. 每个问题必须包含规则依据（引用具体规则内容）
    3. 输出格式：
    - 问题1：...[问题描述]
      依据规则：[规则内容]（来源：[来源表格]）
    - 问题2：...
    
    质控规则：
    {context}
    
    待检查病历：
    {question}
    
    请使用中文输出，确保分析专业严谨。"""
    
    # 创建提示模板
    custom_prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    # 构建检索链
    return (
        {"context": vector_store.as_retriever(search_kwargs={"k": 5}),  # 检索前5条相关规则
         "question": RunnablePassthrough()}
        | custom_prompt
        | llm
        | StrOutputParser()
    )

# 2. 初始化QA链
qa_chain = create_qa_chain(vector_store, llm)

# 3. 定义病历处理函数
def analyze_medical_record(record: str) -> Tuple[List[str], str]:
    """
    分析病历并返回结构化和原始结果
    
    :param record: 输入病历文本
    :return: (问题列表, 完整分析报告)
    """
    # 执行分析
    analysis = qa_chain.invoke(record)
    
    # 后处理提取问题列表
    problem_list = []
    current_problem = ""
    for line in analysis.split('\n'):
        if line.startswith('- 问题'):
            if current_problem:
                problem_list.append(current_problem.strip())
            current_problem = line[2:]  # 去除前缀
        elif line.startswith('  依据规则'):
            current_problem += "\n" + line.strip()
        elif current_problem:
            current_problem += " " + line.strip()
    
    if current_problem:
        problem_list.append(current_problem.strip())
    
    return problem_list, analysis

test_record = """
    入院评估单
患者：张三，男，68岁，入院时间：2024-02-20 12：00
记录时间：23：00
主诉：持续性胸痛3小时，伴冷汗
入院诊断：急性前壁心肌梗死
    """
    
# 执行分析
problems, raw = analyze_medical_record(test_record)

# 打印结构化结果
print("\n结构化结果：")
for i, problem in enumerate(problems):
    print(f"{problem}")
print("\n完整分析报告：")
print(raw)

INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"



结构化结果：
问题1：入院评估单缺少多项必填项（如科室、就诊医院、病历名称、住院号、ID号等基础信息及各项评估分值）   依据规则：[必填项：科室,就诊医院,病历名称,住院号,姓名,性别,年龄,ID号,焦虑,焦虑自评量表分值...（共61项）]（来源：[A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，完整性]）
问题2：入院评估单未记录患者生命体征数据（体温、脉搏、呼吸、血压等）   依据规则：[必填项包含体温(℃)、脉搏(次/min)、呼吸(次/min)、收缩压(mmHg)、舒张压(mmHg)]（来源：[同规则表格中患者转运交接记录单>=3岁必填项]）
问题3：未完成专科评估（压力性损伤、跌倒/坠床等）   依据规则：[必填项包含压力性损伤评估（Braden）总分,跌倒/坠床评估总分等专科评分]（来源：[入院评估单大于3岁必填项]）
问题4：缺少中医特色评估内容（舌象、脉象等）   依据规则：[必填项包含舌质,舌苔,切脉等中医项目]（来源：[同入院评估单规则]）
问题5：未记录患者基础生理参数（身高、体重）   依据规则：[必填项包含身高(cm),体重(kg)]（来源：[同入院评估单规则]）  注：由于待检查病历仅提供片段信息，实际缺失项可能更多。建议对照完整规则逐项核查61项必填内容，尤其注意评估类项目的分值和签名、记录时间等关键要素。

完整分析报告：
根据提供的病历质控规则和待检查病历，分析如下：

- 问题1：入院评估单缺少多项必填项（如科室、就诊医院、病历名称、住院号、ID号等基础信息及各项评估分值）  
依据规则：[必填项：科室,就诊医院,病历名称,住院号,姓名,性别,年龄,ID号,焦虑,焦虑自评量表分值...（共61项）]（来源：[A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，完整性]）

- 问题2：入院评估单未记录患者生命体征数据（体温、脉搏、呼吸、血压等）  
依据规则：[必填项包含体温(℃)、脉搏(次/min)、呼吸(次/min)、收缩压(mmHg)、舒张压(mmHg)]（来源：[同规则表格中患者转运交接记录单>=3岁必填项]）

- 问题3：未完成专科评估（压力性损伤、跌倒/坠床等）  
依据规则：[必填项包含压力性损伤评估（Braden）总分,跌倒/坠床评估总分等专科评分]（来源：[入院